In [2]:
import pandas as pd
import numpy as np

def merge_data():
    # Load all the dataframes and rename common columns
    df_eeg = pd.read_csv(f'{i}_EEG.csv').sort_values('UnixTime').rename(columns={'Row': 'Row_EEG', 'Timestamp': 'Timestamp_EEG', 'QuestionKey': 'QuestionKey_EEG'})
    df_eye = pd.read_csv(f'{i}_EYE.csv').sort_values('UnixTime').rename(columns={'Row': 'Row_EYE', 'Timestamp': 'Timestamp_EYE', 'QuestionKey': 'QuestionKey_EYE'})
    df_gsr = pd.read_csv(f'{i}_GSR.csv').sort_values('UnixTime').rename(columns={'Row': 'Row_GSR', 'Timestamp': 'Timestamp_GSR', 'QuestionKey': 'QuestionKey_GSR'})
    df_ivt = pd.read_csv(f'{i}_IVT.csv').sort_values('UnixTime').rename(columns={'Row': 'Row_IVT', 'Timestamp': 'Timestamp_IVT', 'QuestionKey': 'QuestionKey_IVT'})
    df_tiva = pd.read_csv(f'{i}_TIVA.csv').sort_values('UnixTime').rename(columns={'Row': 'Row_TIVA', 'Timestamp': 'Timestamp_TIVA', 'QuestionKey': 'QuestionKey_TIVA'})

    # Start with EYE data
    merged_df = df_eye

    # Drop 'Elements' from EEG before merging
    if 'Elements' in df_eeg.columns:
        df_eeg = df_eeg.drop('Elements', axis=1)

    # Sequentially merge the dataframes on UnixTime
    merged_df = pd.merge_asof(merged_df, df_ivt, on='UnixTime', direction='nearest')
    merged_df = pd.merge_asof(merged_df, df_eeg, on='UnixTime', direction='nearest')
    merged_df = pd.merge_asof(merged_df, df_tiva, on='UnixTime', direction='nearest')
    merged_df = pd.merge_asof(merged_df, df_gsr, on='UnixTime', direction='nearest')

    # Drop unnecessary columns
    drop_cols = [
        'Battery','Elements','Saccade Index','Saccade Index by Stimulus',
        'Saccade Start','Saccade End','Saccade Duration','Saccade Peak Velocity',
        'Fixation Index','Fixation Index by Stimulus','routineStart',
        'routineStamp','routineEnd','Cat2FeedbackTime','QuestionNumber','Key',
    ]
    
    merged_df = merged_df.drop(columns=[c for c in drop_cols if c in merged_df.columns])
    
    # Fill NaN values in numeric columns with the mean of the column
    numeric_col_names = merged_df.select_dtypes(include=np.number).columns.tolist()
    
    for col in numeric_col_names:
        merged_df[col] = merged_df[col].fillna(merged_df[col].mean())
    
    return merged_df



In [13]:
import pandas as pd
import numpy as np

def merge_data1(i):
    i = str(i)
    
    # IVT
    df_ivt = pd.read_csv(f'{i}_IVT.csv').sort_values('UnixTime')

    # EYE
    df_eye = pd.read_csv(f'{i}_EYE.csv').sort_values('UnixTime')

    # EEG
    df_eeg = pd.read_csv(f'{i}_EEG.csv').sort_values('UnixTime')
    if 'Elements' in df_eeg.columns:
        df_eeg = df_eeg.drop('Elements', axis=1)

    df_tiva = pd.read_csv(f'{i}_TIVA.csv').sort_values('UnixTime')

    # EYE + IVT
    merged_df = pd.merge_asof(df_eye, df_ivt, on='UnixTime', direction='nearest')
    merged_df = merged_df.sort_values('UnixTime')

    # Merged + EEG
    merged_df = pd.merge_asof(merged_df, df_eeg, on='UnixTime', direction='nearest')

    # Merged + TIVA
    if "QuestionKey_x" in merged_df.columns and "QuestionKey_TIVA" in df_tiva.columns:
        merged_df = pd.merge(
            merged_df, df_tiva, left_on="QuestionKey_x", right_on="QuestionKey_TIVA", how="left"
        )

    # Drop unnecessary columns
    drop_cols = [
        'Battery','Elements','Saccade Index','Saccade Index by Stimulus',
        'Saccade Start','Saccade End','Saccade Duration','Saccade Peak Velocity',
        'Fixation Index','Fixation Index by Stimulus','routineStart',
        'routineStamp','routineEnd','Cat2FeedbackTime','Row_x','QuestionKey_y',
        'QuestionNumber','Key','Timestamp_x','QuestionKey_x','Timestamp_y','Row_y'
    ]
    merged_df = merged_df.drop(columns=[c for c in drop_cols if c in merged_df.columns])
    
    # Fill NaN values in numeric columns with the mean of the column
    numeric_col_names = merged_df.select_dtypes(include=np.number).columns.tolist()
    for col in numeric_col_names:
        merged_df[col] = merged_df[col].fillna(merged_df[col].mean())
    
    # Filter out rows with "Nocat" and drop remaining NaNs
    if 'QuestionKey' in merged_df.columns:
        merged_df = merged_df[merged_df["QuestionKey"] != 'Nocat']
    
    merged_df = merged_df.dropna()
    
    return merged_df

In [14]:
l = []

In [15]:
for i in range (1,10):
    l.append(merge_data1(i))

for i in range (11,39):
    l.append(merge_data1(i))


C:\Users\Vishu\AppData\Local\Temp\ipykernel_4340\3624724343.py:14: DtypeWarning: Columns (2,40) have mixed types. Specify dtype option on import or set low_memory=False.
  df_eeg = pd.read_csv(f'{i}_EEG.csv').sort_values('UnixTime')
C:\Users\Vishu\AppData\Local\Temp\ipykernel_4340\3624724343.py:14: DtypeWarning: Columns (2,40) have mixed types. Specify dtype option on import or set low_memory=False.
  df_eeg = pd.read_csv(f'{i}_EEG.csv').sort_values('UnixTime')
C:\Users\Vishu\AppData\Local\Temp\ipykernel_4340\3624724343.py:8: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ivt = pd.read_csv(f'{i}_IVT.csv').sort_values('UnixTime')
C:\Users\Vishu\AppData\Local\Temp\ipykernel_4340\3624724343.py:11: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_eye = pd.read_csv(f'{i}_EYE.csv').sort_values('UnixTime')
C:\Users\Vishu\AppData\Local\Temp\ipykernel_4340\3624724343.py:14: DtypeWar

In [16]:
df = pd.concat(l,axis =0)

In [53]:
new_df = merge_data()

C:\Users\Vishu\AppData\Local\Temp\ipykernel_6820\3518763042.py:6: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_eeg = pd.read_csv(f'{i}_EEG.csv').sort_values('UnixTime').rename(columns={'Row': 'Row_EEG', 'Timestamp': 'Timestamp_EEG', 'QuestionKey': 'QuestionKey_EEG'})


In [17]:
df

,UnixTime,ET_GazeLeftx,ET_GazeLefty,ET_GazeRightx,ET_GazeRighty,ET_PupilLeft,ET_PupilRight,ET_TimeSignal,ET_DistanceLeft,ET_DistanceRight,...,Accelerometer_Y,Accelerometer_Z,Gyro_X,Gyro_Y,Gyro_Z,HeadBandOn,HSI_TP9,HSI_AF7,HSI_AF8,HSI_TP10
12574,1.680003e+09,928.0,539.0,914.0,550.0,-1.000000,-1.000000,104799.253,583.498901,579.684082,...,0.153442,0.997986,-0.433655,2.743988,4.149628,1.0,1.0,1.0,1.0,1.0
12575,1.680003e+09,919.0,548.0,916.0,549.0,-1.000000,-1.000000,104807.563,583.273499,579.813477,...,0.153442,0.997986,-0.433655,2.743988,4.149628,1.0,1.0,1.0,1.0,1.0
12576,1.680003e+09,918.0,549.0,923.0,548.0,2.979706,2.958359,104816.135,583.058960,579.947937,...,0.153442,0.997986,-0.433655,2.743988,4.149628,1.0,1.0,1.0,1.0,1.0
12577,1.680003e+09,911.0,551.0,918.0,547.0,-1.000000,-1.000000,104824.878,582.722351,579.970154,...,0.153381,0.998718,-0.560760,2.392578,4.119720,1.0,1.0,1.0,1.0,1.0
12578,1.680003e+09,918.0,544.0,930.0,540.0,-1.000000,-1.000000,104832.678,582.476868,580.091187,...,0.153381,0.998718,-0.560760,2.392578,4.119720,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111964,1.676984e+09,-1.0,-1.0,1603.0,548.0,-1.000000,2.529755,932978.578,-1.000000,540.473267,...,0.145508,0.997803,-1.136475,2.108459,3.985138,1.0,1.0,1.0,1.0,1.0
111965,1.676984e+09,1612.0,529.0,1603.0,545.0,-1.000000,-1.000000,932986.894,561.086975,540.332153,...,0.145508,0.997803,-1.136475,2.108459,3.985138,1.0,1.0,1.0,1.0,1.0
111966,1.676984e+09,1604.0,544.0,1605.0,551.0,-1.000000,-1.000000,932995.216,561.047791,540.314087,...,0.145508,0.997803,-1.136475,2.108459,3.985138,1.0,1.0,1.0,1.0,1.0
111967,1.676984e+09,-1.0,-1.0,1601.0,545.0,-1.000000,2.524933,933003.720,-1.000000,540.301270,...,0.145508,0.997803,-1.136475,2.108459,3.985138,1.0,1.0,1.0,1.0,1.0


In [18]:
df.to_csv("Final_Dataset.csv")